In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import pulp
import numpy as np
import pandas as pd
import time
import warnings

from itertools import product
from statistics import stdev, mean

from utils import *

In [4]:
class VanillaSolution:
    def __init__(self, problem: Problem):
        self.problem = problem
        self.formula, self.X, self.y = self._formulate()
        self.result = self._solve()


    def _formulate(self):
        n = self.problem.n
        B = self.problem.B
        s = self.problem.s

        X = np.empty((n, n), dtype=object)
        for i, j in product(range(n), repeat=2):
            X[i, j] = pulp.LpVariable(f"X({i},{j})", cat="Binary")

        y = np.array([pulp.LpVariable(f"y({j})", cat="Binary") for j in range(n)])

        formula = pulp.LpProblem()
        formula += pulp.lpSum(y)
        for i in range(n):
            formula += pulp.lpSum(X[i]) == 1, f"アイテム{i}は必ずいづれかの箱に入る"
        for j in range(n):
            formula += pulp.lpDot(s, X[:, j]) <= B * y[j], f"箱{j}のサイズよりもそれに入るアイテムのサイズが小さい"
        
        return formula, X, y
    

    def _solve(self):
        start_time = time.time()
        self.formula.solve(pulp.PULP_CBC_CMD(msg = False))
        solve_time = time.time() - start_time

        if pulp.LpStatus[self.formula.status] != "Optimal":
            warnings.warn("最適解は得られていません。")
        
        convert_value = np.vectorize(lambda x: x.value())
        X_int = convert_value(self.X).astype(int)

        records = []
        for j in range(len(self.y)):
            record = [str(j)] + (X_int[:, j] * self.problem.s).tolist()
            records.append(record)
        allocation = pd.DataFrame.from_records(records, columns=["bin"] + [f"item({i})" for i in range(self.problem.n)])

        return {
            "status": pulp.LpStatus[self.formula.status],
            "objective": self.formula.objective.value(),
            "allocation": allocation,
            "solve_time_sec": solve_time
        }

In [5]:
solution = VanillaSolution(ProblemTest)
draw_allocation(solution.result)

In [6]:
problems = create_problems(10, 8)
solutions = [VanillaSolution(problem) for problem in problems]

time_arr = list(map(lambda x: x.result["solve_time_sec"], solutions))
print(f"mean: {mean(time_arr):.3} s\nstd: {stdev(time_arr):.3} s")

mean: 0.0409 s
std: 0.057 s


In [7]:
print(problems[0].s)
draw_allocation(solutions[0].result)

[5 8 6 1 4 4 4 8]
